----

----

### Setup

----

In [1]:
# You can install v1.15 tensorflow but we recommend running the notebook on colab as its current default is 1.x
# Be careful with python versions, keras and tensorflow!
#!pip install q tensorflow==2.2.4
#!pip install q keras==1.15

In [2]:
import matplotlib.pyplot as plt
import time
import itertools
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from collections import deque

import numpy as np
import virl
import sys
import os
import random
from collections import namedtuple
import collections
import copy

# Import the open AI gym
import gym
import tensorflow as tf
import keras
from keras.models import Sequential
#from keras.layers import Dense
from keras.optimizers import Adam
#from keras import backend as K

from keras import layers
from keras.models import Model
from keras import backend as K
from keras import utils as np_utils
from keras import optimizers
from keras import initializers

from gym.envs.toy_text import discrete


print(sys.version)
print(tf.__version__)
print(keras.__version__)

Using TensorFlow backend.


3.6.12 |Anaconda, Inc.| (default, Sep  9 2020, 00:29:25) [MSC v.1916 64 bit (AMD64)]
1.15.0
2.2.4


-----

### Define a few helper functions 

In [3]:
# define a couple of helper functions
import pandas as pd
EpisodeStats = namedtuple("Stats",["episode_lengths", "episode_rewards"])

def plot_episode_stats(stats, smoothing_window,environment,noisy, noshow=False):

    # Plot the episode reward over time,so we can see when did it converge
    fig1 = plt.figure(figsize=(10,5))
    rewards_smoothed = pd.Series(stats.episode_rewards).rolling(smoothing_window, min_periods=smoothing_window).mean()
    plt.plot(rewards_smoothed)
    plt.xlabel("Episode")
    plt.ylabel("Episode Reward (Smoothed)")
    plt.title("Episode Reward over Time (Smoothed over window size {})".format(smoothing_window))
    plt.savefig('graphs/ps_tm/converge graph/average performance vs number of episodes'+ "for ps_tm agent" + " " + environment + " " + noisy + " "+ '.png')
    if noshow:
        plt.close(fig1)
    else:
        plt.show(fig1)
    return fig1

----

### Define Policy Estimator


In [4]:
def quantizing(state):
    bins = np.array([0e+8, 0.5e+8, 1e+8, 1.5e+8, 2e+8, 2.5e+8,3e+8,3.5e+8,4e+8,4.5e+8,5e+8,5.5e+8])
    index = np.digitize(state, bins) - 1
    return index[0] + index[1] * 12 + index[2] * 12 * 12 + index[3] * 12 * 12 * 12

In [5]:
class NeuralNetworkPolicyEstimator():
    """ 
    A very basic MLP neural network approximator and estimator for poliy search    
    
    The only tricky thing is the traning/loss function and the specific neural network arch
    """
    
    def __init__(self, alpha, n_actions, d_states, nn_config, verbose=False):        
        self.alpha = alpha    
        self.nn_config = nn_config                   
        self.n_actions = n_actions        
        self.d_states = d_states
        self.verbose=verbose # Print debug information        
        self.n_layers = len(nn_config)  # number of hidden layers        
        self.model = []
        self.__build_network(d_states, n_actions)
        self.__build_train_fn()
             

    def __build_network(self, input_dim, output_dim):
        """Create a base network usig the Keras functional API"""
        self.X = layers.Input(shape=(input_dim,))
        net = self.X
        for h_dim in self.nn_config:
            net = layers.Dense(h_dim)(net)
            net = layers.Activation("relu")(net)
        net = layers.Dense(output_dim, kernel_initializer=initializers.Zeros())(net)
        net = layers.Activation("softmax")(net)
        self.model = Model(inputs=self.X, outputs=net)

    def __build_train_fn(self):
        """ Create a custom train function
        It replaces `model.fit(X, y)` because we use the output of model and use it for training.        
        Called using self.train_fn([state, action_one_hot, target])`
        which would train the model. 
        
        Hint: you can think of K. as np.
        
        """
        # predefine a few variables
        action_onehot_placeholder   = K.placeholder(shape=(None, self.n_actions),name="action_onehot") # define a variable
        target                      = K.placeholder(shape=(None,), name="target") # define a variable       
        
        # this part defines the loss and is very important!
        action_prob        = self.model.output # the outlout of the neural network        
        action_selected_prob        = K.sum(action_prob * action_onehot_placeholder, axis=1) # probability of the selcted action        
        log_action_prob             = K.log(action_selected_prob) # take the log
        loss = -log_action_prob * target # the loss we are trying to minimise
        loss = K.mean(loss)
        
        # defining the speific optimizer to use
        adam = optimizers.Adam(lr=self.alpha)# clipnorm=1000.0) # let's use a kereas optimiser called Adam
        updates = adam.get_updates(params=self.model.trainable_weights,loss=loss) # what gradient updates to we parse to Adam
            
        # create a handle to the optimiser function    
        self.train_fn = K.function(inputs=[self.model.input,action_onehot_placeholder,target],
                                   outputs=[],
                                   updates=updates) # return a function which, when called, takes a gradient step
      
    
    def predict(self, s, a=None):              
        if a==None:            
            return self._predict_nn(s)
        else:                        
            return self._predict_nn(s)[a]
        
    def _predict_nn(self,state_hat):                          
        """
        Implements a basic MLP with tanh activations except for the final layer (linear)               
        """                
        x = self.model.predict(state_hat)                                                    
        return x
  
    def update(self, states, actions, target):  
        """
            states: a interger number repsenting the discrete state
            actions: a interger number repsenting the discrete action
            target: a real number representing the discount furture reward, reward to go
            
        """
        action_onehot = np_utils.to_categorical(actions, num_classes=self.n_actions) # encodes the state as one-hot
        self.train_fn([states, action_onehot, target]) # call the custom optimiser which takes a gradient step
        return 
        
    def new_episode(self):        
        self.t_episode  = 0.    

### Define Reinforce Learning

In [6]:
def reinforce(env, estimator_policy, num_episodes, discount_factor=0.9):
    """
    REINFORCE (Monte Carlo Policy Gradient) Algorithm. Optimizes the policy
    function approximator using policy gradient.
    
    Args:
        env: OpenAI environment.
        estimator_policy: Policy Function to be optimized         
        num_episodes: Number of episodes to run for
        discount_factor: reward discount factor
    
    Returns:
        An EpisodeStats object with two numpy arrays for episode_lengths and episode_rewards.
    
    Adapted from: https://github.com/dennybritz/reinforcement-learning/blob/master/PolicyGradient/CliffWalk%20REINFORCE%20with%20Baseline%20Solution.ipynb
    """

    # Keeps track of useful statistics
    stats = EpisodeStats(
        episode_lengths=np.zeros(num_episodes),
        episode_rewards=np.zeros(num_episodes))    
    
    Transition = collections.namedtuple("Transition", ["state", "action", "reward", "next_state", "done"])
    
    for i_episode in range(num_episodes):
        # Reset the environment and pick the fisrst action
        state = env.reset()
        
        episode = []
        
        # One step in the environment
        for t in itertools.count():
            
            # Take a step
            state_oh = np.zeros((1,12**4))
            state_oh[0,quantizing(state)] = 1.0                                  
            action_probs = estimator_policy.predict(state_oh)
            action_probs = action_probs.squeeze()
            
            action = np.random.choice(np.arange(len(action_probs)), p=action_probs)
            
            ##
            next_state, reward, done, _ = env.step(action)
            
            # Keep track of the transition
            episode.append(Transition(
              state=state, action=action, reward=reward, next_state=next_state, done=done))
            
            # Update statistics
            stats.episode_rewards[i_episode] += reward
            stats.episode_lengths[i_episode] = t
            
            # Print out which step we're on, useful for debugging.
            print("\rStep {} @ Episode {}/{} ({})".format(
                    t, i_episode + 1, num_episodes, stats.episode_rewards[i_episode - 1]), end="")            

            if done:
                break
                
            state = next_state
    
        # Go through the episode, step-by-step and make policy updates (note we sometime use j for the individual steps)
        estimator_policy.new_episode()
        new_theta=[]
        for t, transition in enumerate(episode):                 
            state_oh = np.zeros((1,12**4))
            state_oh[0,quantizing(transition.state)] = 1.0 
            
            # The return, G_t, after this timestep; this is the target for the PolicyEstimator
            G_t = sum(discount_factor**i * t.reward for i, t in enumerate(episode[t:]))
           
            # Update our policy estimator
            estimator_policy.update(state_oh, transition.action,np.array(G_t))            
         
    return stats,estimator_policy

----

### Running the traning process

In [7]:
def show_action(actions):
  outcome = "\n"
  for i in range (len(actions)):
    outcome += ("week " + str('{:2}'.format(i+1)) + " : ")
    if actions[i] == 0:
      outcome += (" None              ")
    if actions[i] == 1:
      outcome += (" Full Lockdown     ")
    if actions[i] == 2:
      outcome += (" Track & Trace     ")
    if actions[i] == 3:
      outcome += (" Social Distancing ")
    if len(actions) - 1 != i:
      outcome += (" -> ")
    if (i+1) % 4 == 0:
      outcome += "\n"
  return outcome

In [8]:
def run_one_episode(env,policy_estimator,environment,noisy):
    states = []
    rewards = []
    actions = []
    done = False
    s = env.reset()
    states.append(s)
    while not done:
        state_oh = np.zeros((1,12**4))
        state_oh[0,quantizing(s)] = 1.0                                  
        action_probs = policy_estimator.predict(state_oh)
        action_probs = action_probs.squeeze()

        action = np.random.choice(np.arange(len(action_probs)), p=action_probs)
        next_state, reward, done, _ = env.step(action)

        states.append(next_state)
        rewards.append(reward)
        actions.append(action)
    fig, axes = plt.subplots(1, 2, figsize=(20, 8))
    print(show_action(actions))
    labels = ['s[0]: susceptibles', 's[1]: infectious', 's[2]: quarantined', 's[3]: recovereds']
    states = np.array(states)
    for i in range(4):
        axes[0].set_title("The states analysis for" + " " + environment + " " + noisy + " " + "for ps_tm")
        axes[0].plot(states[:,i], label=labels[i]);
        axes[0].set_xlabel('weeks since start of epidemic')
        axes[0].set_ylabel('State s(t)')
        axes[0].legend()
        axes[1].plot(rewards);
        axes[1].set_title('The Reward to each action' + " " + environment + " " + noisy + " " + "for ps_tm")
        axes[1].set_xlabel('weeks since start of epidemic')
        axes[1].set_ylabel('reward r(t)')
    
    print('total reward', np.sum(rewards))
    plt.savefig('graphs/ps_tm/state_and_reward/state and reward for'+ " " + environment + " " + noisy + " " + "for ps_tm" + '.png')
    return actions

In [9]:
def policy_search_train_and_predict(env,num_episodes,environment,noisy):
    # Instantiate a FunctionApproximator (i.e. the linear function approximator)


    tf.reset_default_graph()

    global_step = tf.Variable(0, name="global_step", trainable=False)
    #policy_estimator = trained_policy_estimator
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        alpha = 0.001  
        n_actions = env.action_space.n
        n_states = 12**4
        nn_config = []

        policy_estimator = NeuralNetworkPolicyEstimator(alpha, n_actions, n_states, nn_config)
        stats,estimator = reinforce(env, policy_estimator,num_episodes, discount_factor=0.97)
        ## then,we use the optimized function approximator to predict on optimal policy and gain them
        actions = run_one_episode(env,estimator,environment,noisy)
        #print(show_action(actions))
    return stats

In [10]:
def overall_training_and_evaluation(num_episodes):
    
    stats_for_problems_nnoise = []
    for i in range (10):
        env = virl.Epidemic(problem_id=i,noisy=False)
        print("This is problem" + " " + str(i) + " " + "without noise for ps_tm")
        stats0 = policy_search_train_and_predict(env,num_episodes,"problem" + " " + str(i),"without noise")
        stats_for_problems_nnoise.append(stats0)
        
    stats_for_problems_noise = []
    for j in range (10):
        env = virl.Epidemic(problem_id=j,noisy=True)
        print("This is problem" + " " + str(j) + " " + "with noise for ps_tm")
        stats1 = policy_search_train_and_predict(env,num_episodes,"problem" + " " + str(j),"with noise")
        stats_for_problems_noise.append(stats1)
        

    env = virl.Epidemic(stochastic=True,noisy=False)
    print("This is stochastic problem " +  " " + "without noise for ps_tm")
    stats2 = policy_search_train_and_predict(env,num_episodes,"stochastic","without noise")
    
    env = virl.Epidemic(stochastic=True,noisy=True)
    print("This is stochastic problem " +  " " + "with noise for ps_tm")
    stats3 = policy_search_train_and_predict(env,num_episodes,"stochastic","with noise")
    
    print("----------------------------------Learning Curves----------------------------------------------")
    return stats_for_problems_nnoise,stats_for_problems_noise,stats2,stats3

In [11]:
def show_graphs_policy_pstm(stats_for_problems_nnoise,stats_for_problems_noise,stats2,stats3):
    '''Used for ploting the converging curve for each training problem and environments'''
    
    for i in range(len(stats_for_problems_nnoise)):
        plot_episode_stats(stats_for_problems_nnoise[i],100,"problem" + " " + str(i),"without noise")
        print("This is the convergence graphs for problem" + " " + str(i) + " " + "without noise for ps_tm")
    for j in range(len(stats_for_problems_noise)):
        plot_episode_stats(stats_for_problems_noise[j],100,"problem" + " " + str(j),"with noise")
        print("This is the convergence graphs for problem" + " " + str(j) + " " + "with noise for ps_tm")
        
    plot_episode_stats(stats2,100,"stochastic","without noise")
    print("This is the convergence graphs for stochastic environment" +  " " + "without noise for ps_tm")
    
    plot_episode_stats(stats3,100,"stochastic","with noise")
    print("This is the convergence graphs for stochastic environment" +  " " + "with noise for ps_tm")

In [12]:
def visualization_for_policySearch_tm():
    # Train and evaluate as well as drawing state graph and reward graph
    stats_for_problems_nnoise,stats_for_problems_noise,stats2,stats3 = overall_training_and_evaluation(2000)
    # Plot the converging curve
    show_graphs_policy_pstm(stats_for_problems_nnoise,stats_for_problems_noise,stats2,stats3)

In [13]:
#visualization_for_policySearch_tm()